### Example of FactScore Baseline (prompt)

In [ ]:
import os
from pathlib import Path
import sys
import json

parent_dir = str(Path(os.getcwd()).resolve().parent)
sys.path.insert(0, parent_dir)

from src.fact_reasoner.context_retriever import ContextRetriever
from src.fact_reasoner.atom_extractor import AtomExtractor
from src.fact_reasoner.atom_reviser import AtomReviser
from src.fact_reasoner.baselines.factscore import FactScore

In [ ]:
# Define the model and backend
model_id = "llama-3.3-70b-instruct"
backend = "rits"
cache_dir = None

# Define the input file (change to ...)
input_file = "/home/radu/git/FactReasoner/examples/flaherty_wikipedia.json"

# Create the retriever, atomizer and reviser.
context_retriever = ContextRetriever(service_type="google", top_k=5, cache_dir=cache_dir)
atom_extractor = AtomExtractor(model_id=model_id, backend=backend)
atom_reviser = AtomReviser(model_id=model_id, backend=backend)

# Create the FactScore pipeline
pipeline = FactScore(
    context_retriever=context_retriever,
    atom_extractor=atom_extractor,
    atom_reviser=atom_reviser,
    model_id=model_id,
    add_topic=True,
    backend=backend,  # Use RITS for the LLM
)

# Load the problem instance from a file
json_file = input_file
with open(json_file, "r") as f:
    data = json.load(f)

# Load the file (json)
print(f"[FactScore] Initializing pipeline from: {json_file}")
pipeline.from_dict_with_contexts(data)

# Build the scorer
pipeline.build(
    has_atoms=True,
    has_contexts=True,
    decontextualize_atoms=False
)

# Print the results
results = pipeline.score()
print(f"[FactScore] Results: {results}")
print(f"Done.")